In [1]:
from PIL import Image
import torch
import clip
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch import optim

/mnt/2bd0c619-af65-4059-bb97-0110ea804e3b/mubuntum/pytorchenv/pytorchenv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/2bd0c619-af65-4059-bb97-0110ea804e3b/mubuntum/pytorchenv/pytorchenv/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
main_df = pd.read_csv('/home/mayowa/Documents/IU/indiana_reports.csv')  

In [3]:
id_df = pd.read_csv('/home/mayowa/Documents/IU/indiana_projections.csv')  

In [4]:
main_df.loc[:5,["impression"]]

,impression
0,Normal chest x-XXXX.
1,No acute pulmonary findings.
2,"No displaced rib fractures, pneumothorax, or p..."
3,1. Bullous emphysema and interstitial fibrosis...
4,No acute cardiopulmonary abnormality.
5,No acute cardiopulmonary findings.


In [5]:
# Check if column 'impression"' has missing values
has_missing_values = main_df["impression"].isna().any()
print(f"Column impression has missing values: {has_missing_values}")

Column impression has missing values: True


In [6]:
print(main_df["impression"].isna().sum()) 

31


In [7]:
clean_df = main_df.dropna(subset=["impression"])

In [8]:
has_missing_values = clean_df["impression"].isna().any()
print(f"Column impression has missing values: {has_missing_values}")

Column impression has missing values: False


In [9]:
print(clean_df["impression"].isna().sum()) 

0


In [10]:
file_path = '/home/mayowa/Documents/cleaned_reports.csv'

In [47]:
clean_df.to_csv(file_path, index=False)

In [11]:
print(main_df["uid"][0])

1


In [1]:
import torch.nn as nn
from torch import optim
import pandas as pd
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import clip

device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training


class ImageTitleDataset(Dataset):
    def __init__(self, image_csv_path, text_csv_path):
        # Load CSV files
        self.image_df = pd.read_csv(image_csv_path)
        self.text_df = pd.read_csv(text_csv_path)
        
        # both CSVs have a "uid" column for linking
        self.image_path = self.image_df['filename'].tolist()
        self.uid = self.image_df['uid'].tolist()
        
        # Initialize an empty list to store text data
        self.text_data = []

        # Match "uid" values and load "impression" column from text CSV
        for uid in self.uid:
            text_row = self.text_df.loc[self.text_df['uid'] == uid]
            if not text_row.empty:
                self.text_data.append(text_row.iloc[0]['image'])
            else:
                # Handle cases where there is no matching "uid"
                self.text_data.append("")  # or any other suitable placeholder
        # Tokenize text
        self.title = clip.tokenize(self.text_data)


    def __len__(self):
        return len(self.title)

#     def __getitem__(self, idx):
#         image = preprocess(Image.open(f"/home/Documents/IU/images/images_normalized/{self.image_path[idx]}"))
#         #print(image)
#         #title = clip.tokenize(self.title[idx])
#         #print(title)
#         return image, self.title
    
    
    def __getitem__(self, idx):
        image = preprocess(Image.open(f"/home/mayowa/Documents/IU/images/images_normalized/{self.image_path[idx]}"))
        title = self.title[idx]
        return image, title

# Example usage:
# image_csv_path = 'image_data.csv'
# text_csv_path = 'text_data.csv'
# dataset = ImageTitleDataset(image_csv_path, text_csv_path)
# image, title = dataset[0]


/mnt/2bd0c619-af65-4059-bb97-0110ea804e3b/mubuntum/pytorchenv/pytorchenv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/2bd0c619-af65-4059-bb97-0110ea804e3b/mubuntum/pytorchenv/pytorchenv/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Example usage: 
image_csv_path = '/home/mayowa/Documents/IU/indiana_projections.csv'
text_csv_path = '/home/mayowa/Documents/IU/indiana_reports.csv'
dataset = ImageTitleDataset(image_csv_path, text_csv_path)

In [3]:
dataset[0][1]

tensor([49406,    87,  3077, 10563,  2217,   537, 26646, 49407,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0], dtype=torch.int32)

In [4]:
dataset[0][0]

tensor([[[-1.1645, -1.1645, -1.1645,  ..., -1.6171, -1.6171, -1.6025],
         [-1.1645, -1.1791, -1.1791,  ..., -1.6171, -1.6171, -1.6025],
         [-1.1791, -1.1791, -1.1791,  ..., -1.6171, -1.6171, -1.6025],
         ...,
         [-1.1499, -1.1499, -1.1499,  ..., -1.4857, -1.4273, -1.4419],
         [-1.1499, -1.1207, -1.1353,  ..., -1.4565, -1.3981, -1.4127],
         [-1.1207, -1.1061, -1.1207,  ..., -1.4565, -1.4127, -1.3835]],

        [[-1.1068, -1.1068, -1.1068,  ..., -1.5720, -1.5720, -1.5570],
         [-1.1068, -1.1218, -1.1218,  ..., -1.5720, -1.5720, -1.5570],
         [-1.1218, -1.1218, -1.1218,  ..., -1.5720, -1.5720, -1.5570],
         ...,
         [-1.0918, -1.0918, -1.0918,  ..., -1.4369, -1.3769, -1.3919],
         [-1.0918, -1.0617, -1.0767,  ..., -1.4069, -1.3469, -1.3619],
         [-1.0617, -1.0467, -1.0617,  ..., -1.4069, -1.3619, -1.3319]],

        [[-0.8688, -0.8688, -0.8688,  ..., -1.3096, -1.3096, -1.2954],
         [-0.8688, -0.8830, -0.8830,  ..., -1

In [5]:
len(dataset.text_data)

7466

In [6]:
# use your own data
#list_image_path = ['folder/image1.jpg','folder2/image2.jpg'] 
#list_txt = ['description for image1.jpg' , 'description for image2.jpg']
#dataset = image_title_dataset(list_image_path,list_txt)
BATCH_SIZE = 8
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=True) #Define your own dataloader

In [7]:
imagei, texti = next(iter(train_dataloader))

In [8]:
imagei, texti

(tensor([[[[-1.7047e+00, -1.7047e+00, -1.7193e+00,  ..., -1.5587e+00,
            -1.5587e+00, -1.5587e+00],
           [-1.6901e+00, -1.6901e+00, -1.7047e+00,  ..., -1.5587e+00,
            -1.5587e+00, -1.5587e+00],
           [-1.6755e+00, -1.6755e+00, -1.6901e+00,  ..., -1.5587e+00,
            -1.5587e+00, -1.5587e+00],
           ...,
           [-2.0103e-01, -6.9648e-02,  3.3439e-03,  ..., -1.5441e+00,
            -1.5295e+00, -1.5295e+00],
           [-2.0103e-01, -6.9648e-02,  3.3439e-03,  ..., -1.5441e+00,
            -1.5295e+00, -1.5295e+00],
           [-2.0103e-01, -8.4247e-02,  1.7942e-02,  ..., -1.5441e+00,
            -1.5295e+00, -1.5295e+00]],
 
          [[-1.6621e+00, -1.6621e+00, -1.6771e+00,  ..., -1.5120e+00,
            -1.5120e+00, -1.5120e+00],
           [-1.6470e+00, -1.6470e+00, -1.6621e+00,  ..., -1.5120e+00,
            -1.5120e+00, -1.5120e+00],
           [-1.6320e+00, -1.6320e+00, -1.6470e+00,  ..., -1.5120e+00,
            -1.5120e+00, -1.5120e+00],


In [9]:
from tqdm import tqdm

In [ ]:
EPOCH = 1

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 


if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

# add your own code to track the training progress.

for epoch in range(EPOCH):
  for batch in tqdm(train_dataloader):
      optimizer.zero_grad()

      images,texts = batch 
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)
      print(total_loss)


In [26]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_10.pt") #just change to your preferred folder/filename

In [6]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("model_checkpoint/model_10.pt")

# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
#checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
#checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
#checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>